# Pertemuan 8

- Geometric Transformation
    - Warp Affine
    - Rotation
    - Translation
    - Affine Transform
    - Perspective Transform
    - CUDA Implementation

___
### Maximizing Jetson Nano Perfomance

In [ ]:
# sudo nvpmodel -m 0
# sudo jetson_clocks

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# check OpenCV Version

cv2.__version__

___
# 1. Geometric Transformation
- Geometric transformation is to geometrically **transform positions** and **orientation** of an image to another position/orientation following a formula function. ([Source](https://medium.com/analytics-vidhya/computer-vision-series-geometric-transformation-89477a7fc0ab#:~:text=Geometric%20transformation%20is%20to%20geometrically,is%20nothing%20without%20colored%20pixels.))
- OpenCV Provide Geometric Transformation Function like Rotation, Translation, Affine Transformation, etc.<br><br>
<img src="resource/geometric_transformation.jpg" style="width:600px"></img>

___
# 1.1 Image Scaling / Resizing
- Scaling is just **resizing** of the image. 
- OpenCV comes with a function `cv2.resize()` for this purpose. 
- The size of the image can be specified manually, or you can specify the scaling factor. 
- Different interpolation methods are used. 
- Preferable interpolation methods are `cv2.INTER_AREA` for **shrinking** and `cv2.INTER_CUBIC` (slow) & `cv2.INTER_LINEAR` for **zooming**.
- We use the function: `cv2.resize (src, dst, dsize, fx, fy, interpolation)` previously discussed at session 1.

___
# 1.2 Transform Warp Affine
- method `cv2.warpAffine(img, M, (w,h), borderValue)`
- where :
    - `img` : input image 
    - `M` : input matrix (rotation/translation/scale)
    - `(w,h)` : size output image 
    - `borderValue` : background color

___
## 1.2.1 Image Translation
![](resource/translation.png)
- Translation matrix : <br>
![](resource/matrix_translation.png)
- **Negative** values of **tx** will shift the image to the **left**
- **Positive** values of **tx** will shift the image to the **right**
- **Negative** values of **ty** will shift the image **up**
- **Positive** values of **ty** will shift the image **down**    

___


In [ ]:
# load image
img = cv2.imread('lena.jpg')
h, w, c= img.shape

# create Translation Matrix
M = np.float32([[1, 0, 0], 
                [0, 1, -100]])

# Apply Warp Affine
translated = cv2.warpAffine(img, M, (w, h))

# Show result
cv2.imshow("Image Translation", translated)
cv2.waitKey(0)
cv2.destroyAllWindows()

___
## 1.2.2 Rotate Image using Affine Transform
- Matrix rotation : <br>
![](resource/matrix_rotation.png) <br>
<br>
where , <br>
![](resource/matrix_rotation_2.png) <br><br>

- method `cv2.getRotationMatrix2D(center, degre, scale)`
- where :
    - `center` : center of rotation (tuple), c/ (30,30)
    - `degre` : rotation angel
    - `scale` : image scale factor


In [ ]:
# load image
img = cv2.imread("lena.jpg")
h, w, c = img.shape

# create Rotation Matrix
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, 45, 1.0)

# Apply Warp Affine
rotated = cv2.warpAffine(img, M, (w, h))

# show result                      
cv2.imshow("rotated image", rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

____
## IMPLEMENTTAION 1 : Translation + Contour Detection
- make the box at the center of image
![](box.png)

In [ ]:
# read image 
img = cv2.imread("box.png")
h0, w0, c = img.shape 

# convert to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply bianry thresholding
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# find contour
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


# draw contours on image
for i, cnt in enumerate(contours):
    x, y, w, h = cv2.boundingRect(cnt)
    
    # create Translation Matrix
    M = np.float32([[1, 0, w0//2 - (x + w//2) ], 
                    [0, 1, h0//2 - (y + h//2)]])

    # Apply Warp Affine
    img = cv2.warpAffine(img, M, (w0, h0), borderValue=(255,255,255) )

# show Image
plt.figure(figsize=(10,5))
plt.imshow(img[:,:,::-1])

## IMPLEMENTTAION 2 : Rotation + Contour Detection

In [ ]:
# read image 
img = cv2.imread("book.png")
h0, w0, c = img.shape 

# convert to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply bianry thresholding
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# apply eroding
kernel = np.ones((3,3),np.uint8)
eroding = cv2.erode(thresh, kernel, iterations = 2)

# find contour
contours, hierarchy = cv2.findContours(eroding, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


# draw contours on image
for cnt in contours :
    # find contour feature
    area = cv2.contourArea(cnt)
    (x,y), (w,h), angel = cv2.minAreaRect(cnt) # find minimum rectangle enclosing contour

    # filter small contour
    if area < 500 :
        continue

    # filter contour with small area
    extent = area / (w*h)
    if extent > 0.5 : 

        # background removal (create mask from final contour) 
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [cnt], 0, (255,255,255), -1) # apply white color
        result = cv2.bitwise_and(img, img, mask=mask)

        # create Rotation Matrix
        center = (w0 // 2, h0 // 2)
        M = cv2.getRotationMatrix2D(center, angel - 90, 1.0) # angel = angel - 90

        # Apply Warp Affine
        rotated = cv2.warpAffine(result, M, (w0, h0))


# show Image
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.subplot(1,2,2)
plt.imshow(rotated[:,:,::-1])

___
## CUDA Warp Affine (Translation & Rotation)

- method `cv2.cuda.warpAffine(src, M, (w,h), dst, borderValue)`
- where :
    - `src` : input image (GPU Mat)
    - `M` : input matrix (rotation/translation/scale)
    - `(w,h)` : size output image 
    - `dst` : output image (GPU Mat)
    - `borderValue` : background color 

In [ ]:

# load image in Host memory
img = cv2.imread("book.png")
h0, w0, c = img.shape

# GPU memory initialization
img_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
img_GpuMat.create((w0, h0), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
gray_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
gray_GpuMat.create((w0, h0), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
thresh_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
thresh_GpuMat.create((w0, h0), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
erode_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
erode_GpuMat.create((w0, h0), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
mask_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
mask_GpuMat.create((w0, h0), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
result_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
result_GpuMat.create((w0, h0), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
rotated_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
rotated_GpuMat.create((w0, h0), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel

# create CUDA morphological eroding object with kernel 3x3
kernel = np.ones((3,3),np.uint8)
MorhErode = cv2.cuda.createMorphologyFilter(cv2.MORPH_ERODE, cv2.CV_8UC1, kernel, iterations = 1)

# upload to GPU memory
img_GpuMat.upload(img)

# CUDA convert to gray
cv2.cuda.cvtColor(img_GpuMat, cv2.COLOR_BGR2GRAY, gray_GpuMat)

# apply CUDA thresholding
cv2.cuda.threshold(gray_GpuMat, 72, 255, cv2.THRESH_BINARY_INV, thresh_GpuMat)

# Apply CUDA Morphological Erode
MorhErode.apply(thresh_GpuMat, erode_GpuMat)


# Download Matrix to Host Memory                                                               
eroding = erode_GpuMat.download()

# find contour
contours, hierarchy = cv2.findContours(eroding, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contours on image
for cnt in contours :
    # find contour feature
    area = cv2.contourArea(cnt)
    (x,y), (w,h), angel = cv2.minAreaRect(cnt) # find minimum rectangle enclosing contour

    # filter small contour
    if area < 500 :
        continue

    # filter contour with small area
    extent = area / (w*h)
    if extent > 0.5 : 

        # background removal (create mask from final contour) 
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [cnt], 0, (255,255,255), -1) # apply white color

        # Upload mask matrix to GPU Memory
        mask_GpuMat.upload(mask)

        # Apply CUDA bitwise operation
        cv2.cuda.bitwise_not(img_GpuMat, result_GpuMat, mask=mask_GpuMat)
        cv2.cuda.bitwise_not(result_GpuMat, result_GpuMat, mask=mask_GpuMat)

        # create Rotation Matrix
        center = (w0 // 2, h0 // 2)
        M = cv2.getRotationMatrix2D(center, angel - 90, 1.0) # angel = angel - 90

        # Apply CUDA Warp Affine
        cv2.cuda.warpAffine(result_GpuMat, M, (w0, h0), rotated_GpuMat)


# Download Matrix to Host Memory 
rotated = rotated_GpuMat.download()

# show Image
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.subplot(1,2,2)
plt.imshow(rotated[:,:,::-1])

___
# 3. Affine Tranform & Perspective Transform
<img src="resource/TransformationsDifference.png" style="width:500px"></img>


## 3.1 Affine Transform
- $M$ is 2x3 tranfromation matrix to pass throuh `cv2.warpAffine()`
- method `cv.getAffineTransform(pts1, pts2)`
- where :
    - `pts1` :  triangle vertices source image
    - `pts2` :  triangle vertices destination image <br>
<img src="resource/Warp_Affine_Tutorial_Theory_0.jpg" style="width:300px"></img>
- source image :
    - point 0,0
    - point w, 0
    - point 0, h

In [ ]:
# read image
img = cv2.imread('book_standing.jpg')
h, w, c = img.shape 

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convert to binary image
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# find contour
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

for cnt, hrcy in zip(contours, hierarchy[0]) :
    # find contour feature
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # filter small contour
    if area < 200 :
        continue

    # filter contour with small area and doesnt have a parent (hole contour)
    extent = area / (w*h)
    if extent > 0.4 and hrcy[3] != -1:
        cv2.drawContours(img, [cnt], 0, (0,255,0), 2) # apply white color

        # find minimum enclosing object
        perimeter = cv2.arcLength(cnt, True)
        eMin = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
        eMin = np.array(eMin[:,0,:]) # enclosing min point in counter clockwise direction 
                                                     # tr, br, bl, tl

        # draw point on enclosing minimum 
        
        # tl, tr, br, bl
        for pt in eMin : 
            cv2.circle(img, pt, 10, (0,0,255), -1)

        pts1 = np.array([eMin[0], eMin[1], eMin[2], ], np.float32) # bl, tl, tr
        pts2 = np.array([[w,0], [w,h], [0,h]], np.float32)
        M = cv2.getAffineTransform(pts1, pts2)
        result = cv2.warpAffine(img, M, (w, h))

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.subplot(1,2,2)
plt.imshow(result[:,:,::-1])

____
## 3.2 Perspective Transformation

- M is 3x3 transformation matrix for Perspective Transform.
- to create M matrix, we need **4 points on the input image** and corresponding points on the **output image**
- method `cv2.getPerspectiveTransform(pts1, pts2)`
- where :
    - `pts1` : four vertices source image
    - `pts2` : four vertices destination image <br><br>
<img src="resource/perspective.png" style="width:400px"></img> <br><br><br>
- sudoku image problem : <br><br>
<img src="resource/sudoku.png" style="width:400px"></img> 

In [ ]:
# load image
img = cv2.imread('sudoku.jpg')
h, w, c = img.shape

# define four vertices of source & destination image
# tl, tr, br, bl
pts1 = np.float32([[56,65],[368,52],[389,390], [28,387]]) # the corder of sudoku image
pts2 = np.float32([[0,0],[w,0],[w,h],[0,h]]) # the image corner 

# create Perpective Transform Matrix
M = cv2.getPerspectiveTransform(pts1,pts2)

# Apply Warp perspective transform
output = cv2.warpPerspective(img, M, (w,h))

# draw small dot in original image based on `pts1`
for x, y in pts1.astype(np.uint16):
    cv2.circle(img, (x,y), 4, (255, 255, 0), -1)

# display result
cv2.imshow("Original Image", img)
cv2.imshow("Perspective Transform Image", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

___

## IMPLEMENTATION : Affine Transform Implementation to Crop ID Card Photo
<img src="resource/crop_ktp.png" style="width:400px"></img>

In [ ]:
# read image
img = cv2.imread('ktp5.jpg')

# convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convert to binary image
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# find contour
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

for cnt, hrcy in zip(contours, hierarchy[0]) :
    # find contour feature
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # filter small contour
    if area < 200 :
        continue

    # filter contour with small area and doesnt have a parent (hole contour)
    extent = area / (w*h)
    if extent > 0.4 and hrcy[3] != -1:

        (x,y), (w_ma, h_ma), angel = cv2.minAreaRect(cnt)
        box = (x,y), (w_ma, h_ma), angel 
        
        bbox = cv2.boxPoints(box).astype(np.float32) # bbox output in counter clockwise direction point -> bl, tl, tr, br
        bbox = np.array([bbox[1], bbox[2], bbox[3], bbox[0]]) # re-arange bbox in clockwise direction -> tl, tr, br, bl
        
        dst = np.array([[0, 0],
                        [w_ma, 0], 
                        [w_ma, h_ma], 
                        [0, h_ma]], np.float32)

        M = cv2.getPerspectiveTransform(bbox, dst)
        warped = cv2.warpPerspective(img, M, (int(w_ma), int(h_ma)))

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)

plt.imshow(img[:,:,::-1])
plt.subplot(1,2,2)
plt.imshow(warped[:,:,::-1])


___
# Source
- [tutorial_warp_affine](https://docs.opencv.org/master/d4/d61/tutorial_warp_affine.html)
- [py_geometric_transformations](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html)